# Imports

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib
#import urllib3.request,sys,time
from urllib.error import HTTPError
from urllib.request import urlopen
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')

# Fonctions getArticle et getNews

In [2]:
def getArticle(pageurl):
    try:
        html = urlopen(pageurl)
    except HTTPError as e:
        return None
    '''try:
        bsObj = BeautifulSoup(html.read())
        item1 = bsObj.find("ul",{'class':{'metro-mosaic'}}).findAll('h3')
    except AttributeError as e:
        return None'''
    try:
        bsObj = BeautifulSoup(html.read())
        item2 = bsObj.find("div",{'class':{'metro-breaker widget'}}).findAll('h3')
    except AttributeError as e:
        return None
    articles_list=[]
    '''for j in item1:#one item is one article
        article=dict()
        article['url']=j.find('a').get('href')
        articles_list.append(article)'''
    for j in item2:#one item is one article
        article=dict()
        article['url']=j.find('a').get('href')
        articles_list.append(article)
    return articles_list

In [3]:
#crée un dictionnaire avec les infos de l'article
def getNews(article_url):
    try:
        html = urlopen(article_url)
    except HTTPError as e:
        print(article_url)
        return None
    bsObj = BeautifulSoup(html.read())
    article=dict()
    try:
        article["author"]=bsObj.find("span",{'class':{'author-container'}}).text
    except AttributeError as e:
        article["author"]=''
    try:
        article["headline"]=bsObj.find("header",{'class':{'single-article-header'}}).h1.text
    except AttributeError as e:
        article["headline"]=''
    try:
        article["date"]=bsObj.find("span",{'class':{'post-date'}}).text[:17]
    except AttributeError as e:
        article["date"]= ''
    try:
        paras=bsObj.find_all("div",{'class':{'article-body'}})
        fulltext=""
        for para in paras:
            paras_text=" ".join(i.text for i in para.find_all("p"))
            fulltext=fulltext+paras_text
        article["fulltext"]=fulltext
    except AttributeError as e:
        article["fulltext"]=''
    article["url"]=article_url
    return article

In [4]:
#transforme la liste d'url en dataframe avec les articles
def get_articles_from_links(articles_links):
    articles = []
    for ind in articles_links.index :
        if ind%1000==0:
            print(ind)
        article_url = articles_links.iloc[ind,0]
        news=getNews(article_url)
        if news is not None:
            articles.append(news)
        #time.sleep(1)
    return articles

In [39]:
def articles_from_base_url_MET(base_url):
    url = []
    for i in range (1, 92): #On va de 1 en 1 pour Metro. Le rangemax dépend du sujet.
        url.append(base_url+'/'+str(i)+'/')
    articles_links = pd.DataFrame()
    #création de la database avec tous les liens d'articles
    for i in range(len(url)):
        inter = pd.DataFrame(getArticle(url[i]))
        articles_links = pd.concat([articles_links, inter], ignore_index=True, axis=0)
    news = get_articles_from_links(articles_links)
    df_news = pd.DataFrame(news)
    return df_news

# Dataframe Facebook

rangemax : 92

In [40]:
#URL de base
base_url = 'https://metro.co.uk/tag/facebook/page'

In [41]:
df_news_fb = articles_from_base_url_MET(base_url)

0
1000
2000


In [42]:
df_news_fb

,author,headline,date,fulltext,url
0,Vicky Shaw,"Bank warns of scams targeting Facebook, WhatsA...",Friday 5 May 2023,TSB has issued a warning on the scale of socia...,https://metro.co.uk/2023/05/05/bank-warns-of-s...
1,Josh Layton,Magnet fishers reel in ‘record find of more th...,Sunday 30 Apr 202,A team of magnet fishers has laid claim to the...,https://metro.co.uk/2023/04/30/magnet-fishers-...
2,Jessica Kwong,Donald Trump posts on Instagram for first time...,Wednesday 19 Apr,Former President Donald Trump posted on Instag...,https://metro.co.uk/2023/04/19/donald-trump-po...
3,Pierra Willix,"Celine Dion impersonator spent £45,000 for one...",Friday 7 Apr 2023,"Sorry, this video isn't available any more. A ...",https://metro.co.uk/2023/04/07/celine-dion-imp...
4,Metro Tech Reporter,Meta’s new AI model can identify items in imag...,Thursday 6 Apr 20,Facebook-owner Meta has published an artificia...,https://metro.co.uk/2023/04/06/metas-new-ai-mo...
...,...,...,...,...,...
2263,Metrowebukmetro,Facebook fury over adultery billboards,Tuesday 26 Jan 20,Disgusted Facebook users have launched a campa...,https://metro.co.uk/2010/01/26/adultery-billbo...
2264,Ted Thornhill,Ben Kinsella’s Facebook taunts from killer Jad...,Monday 25 Jan 201,"Murdered\nBen Kinsellaâs killer,\nJade Brait...",https://metro.co.uk/2010/01/25/ben-kinsellas-f...
2265,Jo Steele,‘Web suicide site’ gets around Facebook ban,Sunday 24 Jan 201,A website locked in a legal tug-of-war with Fa...,https://metro.co.uk/2010/01/24/web-2-0-suicide...
2266,Ted Thornhill,‘Baby with cigarette’ Facebook photo probed by...,Wednesday 20 Jan,A Facebook photograph of a baby with a cigaret...,https://metro.co.uk/2010/01/20/police-probe-ba...


In [43]:
df_fb_MET = df_news_fb[df_news_fb['url'].str.contains('meta|facebook|messenger|instagram|oculus|whatsApp|zuckerberg|olivan|clegg|social-media|gafa|gafam|big-tech|consumer-privacy|tech')]

In [44]:
df_fb_MET['url']

0       https://metro.co.uk/2023/05/05/bank-warns-of-s...
2       https://metro.co.uk/2023/04/19/donald-trump-po...
3       https://metro.co.uk/2023/04/07/celine-dion-imp...
4       https://metro.co.uk/2023/04/06/metas-new-ai-mo...
7       https://metro.co.uk/2023/03/17/donald-trump-po...
                              ...                        
2262    https://metro.co.uk/2010/02/02/facebook-and-tw...
2263    https://metro.co.uk/2010/01/26/adultery-billbo...
2264    https://metro.co.uk/2010/01/25/ben-kinsellas-f...
2265    https://metro.co.uk/2010/01/24/web-2-0-suicide...
2266    https://metro.co.uk/2010/01/20/police-probe-ba...
Name: url, Length: 1679, dtype: object

# Dataframe Amazon

In [45]:
#URL de base
base_url = 'https://metro.co.uk/tag/amazon/page'

In [46]:
df_news_am = articles_from_base_url_MET(base_url)

0
1000


In [47]:
df_am_MET = df_news_am[df_news_am['url'].str.contains('amazon|prime|whole-foods|zappos|pillpack|twitch|audible|goodreads|imdb|bezos|jassy|olsavsky|alexander|gafa|gafam|big-tech|consumer-privacy|tech')]

In [48]:
yy=df_am_MET[df_am_MET['fulltext'].str.contains('forest|tribe|jungle|swallows-and-amazon|amazons')]
df_am_MET.drop(yy.index,inplace=True)

# Dataframe Apple

In [49]:
#URL de base
base_url = 'https://metro.co.uk/tag/apple/page'

In [50]:
df_news_ap = articles_from_base_url_MET(base_url)

0
1000


In [51]:
df_ap_MET = df_news_ap[df_news_ap['url'].str.contains('apple|iphone|iPad|mac|watch|macbook|ios|airpods|app-store|itunes|icloud|beats|siri|shazam|cook|jobs|gafa|gafam|big-tech|consumer-privacy|tech')]

In [52]:
yy=df_ap_MET[df_ap_MET['fulltext'].str.contains('apples|pie|pies|recipe|tart|crumble|bake|vinegar')]
df_ap_MET.drop(yy.index,inplace=True)

# Dataframe Google

In [53]:
#URL de base
base_url = 'https://metro.co.uk/tag/google/page'

In [54]:
df_news_go = articles_from_base_url_MET(base_url)

0
1000


In [55]:
df_go_MET = df_news_go[df_news_go['url'].str.contains('google|android|chrome|gmail|maps|playstore|pixel|waze|youTube|alphabet|mandiant|fitbit|looker|nest|doubleclick|page|brin|pichai|kurian|gafa|gafam|big-tech|consumer-privacy|tech')]

# Dataframe Microsoft

In [56]:
#URL de base
base_url = 'https://metro.co.uk/tag/microsoft/page'

In [57]:
df_news_mi = articles_from_base_url_MET(base_url)

0


In [58]:
df_mi_MET = df_news_mi[df_news_mi['url'].str.contains('microsoft|windows|cortana|excel|explorer|office|edge|teams|outlook|powerpoint|skype|surface|word|xbox|linkedIn|github|mojang|gates|nadella|gafa|gafam|big-tech|consumer-privacy|tech')]

# Dataframe final

In [59]:
from google.colab import files
df_final = pd.concat([df_fb_MET, df_am_MET, df_ap_MET, df_go_MET, df_mi_MET]).reset_index()
df_final.drop_duplicates(subset ="url", keep = 'first', inplace=True)
df_final.to_csv('articlesMET.csv')
files.download('articlesMET.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>